# Simulacion de rutas 

### Importar librerias

In [38]:
import pandas as pd
import numpy as np
import math
from algoritmoCVRP import  *
import random

In [39]:
!pip install geopy


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Primera parte: Distribucion de probabilidad de numero de pedidos

In [2]:
clientes = pd.read_excel("direcciones.xlsx")
clientes

,ciudad,Colonia,codigo postal,Calle,numero de casa
0,Monterrey,FIERRO MTY,64590,JESUS M GARZA,4026.0
1,Monterrey,VALLE DE SANTA LUCIA MTY,64140,LIC PEDRO ZORRI,404.0
2,Monterrey,JARDINES DEL PAS MTY,64900,AVE FUNDADORES,5300.0
3,Monterrey,CIUDAD NATURA,64989,AV ABRAHAM LIN,4001.0
4,Monterrey,VILLA LOS PINOS MTY,64770,PINO SILVESTRE,2923.0
...,...,...,...,...,...
351,Escobedo,LA ALIANZA,64103,PRIV PACHUCA,178.0
352,Escobedo,SANTA LUZ,66070,OXFORD,102.0
353,Escobedo,BUENA VISTA,66583,C PELICANO,238.0
354,Escobedo,LOS PARQUES RESIDENCIAL,66023,COBALTO,517.0


In [3]:
informacion_compra = pd.read_excel("informacion_compra.xlsx")
informacion_compra

,Producto,Unidades,Factura
0,48443,1,799186
1,42877,1,717106
2,48296,1,468125
3,34704,1,176490
4,47158,2,674550
...,...,...,...
31649,35303,2,778814
31650,37584,1,632798
31651,19418,1,784747
31652,40176,1,73823


In [4]:
l =  informacion_compra[" Unidades "].mean()#lambda: media de numero de pedidos realizados
l

1.1064320465028117

In [5]:
pedidoMaximo = informacion_compra[" Unidades "].max()

[i for i in range(1,pedidoMaximo + 1)]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [6]:
prob = pd.DataFrame({"X": [i for i in range(0,pedidoMaximo + 1)]})


prob["p(x)"] = [ ((l ** x) *  math.exp(-l)) / math.factorial(x) for x in range(0,pedidoMaximo + 1)] 
prob["p(x)"] = prob["p(x)"] / prob["p(x)"].sum()  #normalizamos al dividir entre la suma total de todas las probabilidades

#Probabilidad acumulada
acumul = []
for i in range(0,prob.shape[0]):
    value = prob.loc[0:i, "p(x)"].sum() 
    acumul.append(value)

prob["p(x) acumulada"] = acumul 

#Limite inferior y superior

prob["Lim inferior"] =  [prob["p(x) acumulada"][i] - prob["p(x)"][i]  for i in range(prob.shape[0])]
prob["Lim superior"] =  [prob["p(x) acumulada"][i]  for i in range(prob.shape[0])]


In [7]:
prob

,X,p(x),p(x) acumulada,Lim inferior,Lim superior
0,0,3.307369e-01,0.330737,0.000000,0.330737
1,1,3.659379e-01,0.696675,0.330737,0.696675
2,2,2.024427e-01,0.899118,0.696675,0.899118
3,3,7.466304e-02,0.973781,0.899118,0.973781
4,4,2.065239e-02,0.994433,0.973781,0.994433
5,5,4.570094e-03,0.999003,0.994433,0.999003
6,6,8.427498e-04,0.999846,0.999003,0.999846
7,7,1.332065e-04,0.999979,0.999846,0.999979
8,8,1.842299e-05,0.999997,0.999979,0.999997
9,9,2.264865e-06,1.000000,0.999997,1.000000


### Segunda parte: Simular pedidos de los clientes

In [8]:
numClientes = clientes.shape[0] #Numero de clientes potenciales
numClientes

356

In [9]:
num_pedidos = pd.DataFrame({
    "Cliente": [i for i in range(numClientes)]

})

# Set the "X" column as the index
num_pedidos.set_index("Cliente", inplace=True)

# Reset the index to default integer index and remove the "X" column from the index
num_pedidos.reset_index(drop=True, inplace=True)

#Numero de num_pedidos que pedira cada cliente en base a las probabilidades
numPedidos = []
num_aleatorio = []
for i in range(num_pedidos.shape[0]):
    p = random.uniform(0,1)
    num_aleatorio.append(p)
    for j in range(prob.shape[0]):
        if (prob["Lim inferior"][j] <= p and p < prob["Lim superior"][j]):
            numPedidos.append(j) #Numero de num_pedidos
        else:
            continue

num_pedidos["Aleatorio"] = num_aleatorio
num_pedidos["Numero de pedidos"] = numPedidos

In [10]:
num_pedidos

,Aleatorio,Numero de pedidos
0,0.527082,1
1,0.161062,0
2,0.061546,0
3,0.748310,2
4,0.149171,0
...,...,...
351,0.502875,1
352,0.905077,3
353,0.865957,2
354,0.016917,0


### Parte 3: Designar tipo de pedido por cliente

In [11]:
ventaDia = {}

for idx,producto in enumerate(informacion_compra["Producto"].tolist()):
    if producto not in ventaDia.keys():
        ventaDia[producto] = 1
    else:
        ventaDia[producto] += list(informacion_compra[" Unidades "])[idx]


ventaDia
     

{48443: 43,
 42877: 18,
 48296: 28,
 34704: 45,
 47158: 5,
 1978: 4,
 23723: 75,
 17482: 87,
 5807: 3,
 840: 34,
 34068: 102,
 35440: 66,
 4621: 1,
 47360: 1,
 20650: 5,
 28133: 1,
 18164: 41,
 20254: 16,
 17737: 87,
 37584: 124,
 23248: 14,
 20800: 1,
 17165: 49,
 23566: 12,
 17515: 30,
 38215: 7,
 43296: 14,
 24371: 108,
 34483: 106,
 39526: 6,
 41621: 2,
 37450: 19,
 34386: 111,
 17449: 8,
 27452: 3,
 41643: 32,
 3003: 6,
 672: 2,
 35184: 26,
 9256: 1,
 23308: 8,
 37202: 9,
 42148: 1,
 8290: 8,
 41916: 33,
 34666: 8,
 34112: 126,
 17437: 19,
 47201: 34,
 34327: 23,
 43947: 1,
 44471: 2,
 30213: 104,
 17138: 31,
 17662: 23,
 45156: 5,
 37177: 18,
 34097: 37,
 36065: 15,
 19250: 1,
 40280: 34,
 199: 92,
 47825: 1,
 35303: 25,
 38524: 18,
 47117: 36,
 47993: 20,
 24335: 266,
 34109: 20,
 17259: 76,
 44042: 11,
 35617: 27,
 17193: 68,
 34673: 60,
 37583: 108,
 20190: 11,
 24415: 73,
 23292: 41,
 3067: 1,
 34358: 26,
 24155: 1,
 46866: 9,
 15196: 62,
 24333: 118,
 35221: 63,
 34387: 45,


In [12]:
venta_dia =  pd.DataFrame(list(ventaDia.items()), columns=['Producto', 'Venta promedio/dia'])
venta_dia

,Producto,Venta promedio/dia
0,48443,43
1,42877,18
2,48296,28
3,34704,45
4,47158,5
...,...,...
4756,12733,1
4757,27087,1
4758,23096,1
4759,23991,1


In [13]:
venta_dia["Prob Relativa"] = [venta_dia["Venta promedio/dia"][x] / venta_dia["Venta promedio/dia"].sum() for x in range(venta_dia.shape[0])]

#Probabilidad acumulada
acumul = []
for i in range(0,venta_dia.shape[0]):
    value = venta_dia.loc[0:i, "Prob Relativa"].sum() 
    acumul.append(value)

venta_dia["Prob acumulada"] = acumul 

#Limite inferior y superior

venta_dia["Lim inferior"] =  [venta_dia["Prob acumulada"][i] - venta_dia["Prob Relativa"][i]  for i in range(venta_dia.shape[0])]
venta_dia["Lim superior"] =  [venta_dia["Prob acumulada"][i]  for i in range(venta_dia.shape[0])]


In [14]:
venta_dia

,Producto,Venta promedio/dia,Prob Relativa,Prob acumulada,Lim inferior,Lim superior
0,48443,43,0.001245,0.001245,0.000000,0.001245
1,42877,18,0.000521,0.001766,0.001245,0.001766
2,48296,28,0.000811,0.002576,0.001766,0.002576
3,34704,45,0.001303,0.003879,0.002576,0.003879
4,47158,5,0.000145,0.004024,0.003879,0.004024
...,...,...,...,...,...,...
4756,12733,1,0.000029,0.999884,0.999855,0.999884
4757,27087,1,0.000029,0.999913,0.999884,0.999913
4758,23096,1,0.000029,0.999942,0.999913,0.999942
4759,23991,1,0.000029,0.999971,0.999942,0.999971


### Parte 4: Simular qué tipo de pedido realiza cada cliente

In [15]:
tipo_pedidos = pd.DataFrame({
    "Cliente": [i for i in range(num_pedidos.shape[0])]

})

# Set the "X" column as the index
tipo_pedidos.set_index("Cliente", inplace=True)

# Reset the index to default integer index and remove the "X" column from the index
tipo_pedidos.reset_index(drop=True, inplace=True)

def tipoPedido(num_pedidos,prob_tipoPedido,num_productosTotal):

    tipo_pedido = [0 for i in range(num_productosTotal)]

    for i in range(num_pedidos):

        prob_pedido = random.uniform(0,1)
        #print(prob_pedido)

        for j in range(num_productosTotal):           
            if (prob_tipoPedido["Lim inferior"][j] <= prob_pedido and prob_pedido < prob_tipoPedido["Lim superior"][j]):
                tipo_pedido[j] += 1
            else:
                continue
    
    return tipo_pedido
    

In [16]:

product_dict = {
    48443: 43,
    42877: 18,
    48296: 28,
    34704: 45,
    47158: 5,
    1978: 4,
    23723: 75,
    17482: 87
}

# Sort the dictionary by values (values of the keys)
sorted_keys_by_value = sorted(product_dict, key=lambda k: str(k).zfill(10))

# Create a new dictionary with keys sorted by values
sorted_dict = {key: product_dict[key] for key in sorted_keys_by_value}

# Now, 'sorted_dict' contains the keys sorted by their associated values
print(sorted_dict)

{1978: 4, 17482: 87, 23723: 75, 34704: 45, 42877: 18, 47158: 5, 48296: 28, 48443: 43}


In [17]:
def designarPedidos(num_pedidos,prob_tipoPedido,num_productos):

    numPedidos = list(num_pedidos["Numero de pedidos"]) #lista de numeros de pedidos por cada cliente

    pedidos = {}

    names_products = list(prob_tipoPedido["Producto"])
    for i in range(prob_tipoPedido.shape[0]):
        pedidos[names_products[i]] = []


    for num in numPedidos:
        listaPedidos = tipoPedido(num,prob_tipoPedido,num_productos)
        index = 0
        values = list(pedidos.values())
        for key in pedidos.keys():
            values[index].append(listaPedidos[index]) #numero de pedidos por tipo de pedido
            index += 1
        
        for index,key in enumerate(pedidos):
            pedidos[key] = values[index]
    
    # Sort the dictionary by values (values of the keys)
    sorted_keys_by_value = sorted(pedidos, key=lambda k: str(k).zfill(10))

    # Create a new dictionary with keys sorted by values
    pedidos_sorted = {key: pedidos[key] for key in sorted_keys_by_value}

    pedidosFinal = pd.concat([num_pedidos, pd.DataFrame(pedidos_sorted)], axis=1)
    return pedidosFinal

                
    

In [18]:
productos_porCliente= designarPedidos(num_pedidos,venta_dia,venta_dia.shape[0])
productos_porCliente

,Aleatorio,Numero de pedidos,23,29,30,60,66,67,89,91,...,52910,52919,52925,52937,52940,52942,52984,52993,53014,53475
0,0.527082,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.161062,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.061546,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.748310,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.149171,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,0.502875,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
352,0.905077,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
353,0.865957,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
354,0.016917,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
productos_porCliente.iloc[:,2:productos_porCliente.shape[1]].loc[355,:].value_counts()

0    4760
1       1
Name: 355, dtype: int64

###  Parte 5: Calcular volumen de carga por cada cliente

In [20]:
productos_volumen = pd.read_csv("info_productos.csv")
productos_volumen

,Producto,Volumen mtro3
0,1,0.001050
1,2,0.026710
2,3,0.020000
3,4,0.056515
4,5,0.025238
...,...,...
53627,53628,0.000266
53628,53629,0.000560
53629,53630,0.000540
53630,53631,0.000069


In [21]:
productos_porCliente

,Aleatorio,Numero de pedidos,23,29,30,60,66,67,89,91,...,52910,52919,52925,52937,52940,52942,52984,52993,53014,53475
0,0.527082,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.161062,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.061546,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.748310,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.149171,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,0.502875,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
352,0.905077,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
353,0.865957,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
354,0.016917,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
list(productos_porCliente.iloc[:,2:productos_porCliente.shape[1]].columns)

[23,
 29,
 30,
 60,
 66,
 67,
 89,
 91,
 109,
 118,
 121,
 122,
 125,
 145,
 151,
 158,
 160,
 163,
 166,
 169,
 173,
 181,
 199,
 205,
 209,
 215,
 219,
 229,
 238,
 239,
 290,
 305,
 309,
 315,
 319,
 320,
 326,
 332,
 338,
 341,
 344,
 348,
 354,
 359,
 367,
 369,
 371,
 386,
 398,
 411,
 414,
 422,
 458,
 667,
 670,
 672,
 682,
 690,
 692,
 714,
 716,
 717,
 721,
 722,
 726,
 741,
 764,
 765,
 768,
 777,
 783,
 786,
 792,
 796,
 806,
 828,
 830,
 840,
 841,
 844,
 848,
 849,
 871,
 872,
 882,
 897,
 920,
 922,
 928,
 929,
 937,
 939,
 960,
 961,
 963,
 979,
 980,
 1060,
 1081,
 1086,
 1094,
 1102,
 1106,
 1120,
 1123,
 1144,
 1146,
 1148,
 1151,
 1153,
 1156,
 1159,
 1165,
 1167,
 1184,
 1205,
 1206,
 1207,
 1230,
 1233,
 1253,
 1256,
 1260,
 1285,
 1299,
 1301,
 1306,
 1311,
 1332,
 1335,
 1336,
 1346,
 1351,
 1364,
 1365,
 1367,
 1393,
 1400,
 1415,
 1418,
 1424,
 1425,
 1426,
 1427,
 1428,
 1434,
 1440,
 1441,
 1442,
 1443,
 1447,
 1454,
 1455,
 1458,
 1459,
 1460,
 1464,
 1466,

In [23]:
productosVolumen = {}

for i in range(productos_volumen.shape[0]):
    key = productos_volumen["Producto"][i]
    value = productos_volumen[" Volumen mtro3"][i]
    productosVolumen[key] = value

In [24]:
def calcularVolumen(df,numberProduct_clientes,productos_volumen):

        volumenTotalPorPedidos = []
        for idx,productos in df.iterrows():
                volumen_porCliente = 0
                j = 0
                for numeroDeProductos in productos:
                        try:
                                volumen_porCliente += numeroDeProductos  * productos_volumen[numberProduct_clientes[j]]
                        except:
                                None
                        j += 1

                volumenTotalPorPedidos.append(volumen_porCliente)        



        return volumenTotalPorPedidos      
        


In [25]:
productos_porCliente["Volumen por pedidos (m^3)"] = calcularVolumen(productos_porCliente.iloc[:,2:productos_porCliente.shape[1]],list(productos_porCliente.iloc[:,2:productos_porCliente.shape[1]].columns),productosVolumen)

In [26]:
productos_porCliente

,Aleatorio,Numero de pedidos,23,29,30,60,66,67,89,91,...,52919,52925,52937,52940,52942,52984,52993,53014,53475,Volumen por pedidos (m^3)
0,0.527082,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.017079
1,0.161062,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
2,0.061546,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
3,0.748310,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.290464
4,0.149171,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,0.502875,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000957
352,0.905077,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.174686
353,0.865957,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.276000
354,0.016917,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [27]:
#volumen total entre todos los clientes
productos_porCliente["Volumen por pedidos (m^3)"].sum()

53.439137689999995

In [28]:
clientes

,ciudad,Colonia,codigo postal,Calle,numero de casa
0,Monterrey,FIERRO MTY,64590,JESUS M GARZA,4026.0
1,Monterrey,VALLE DE SANTA LUCIA MTY,64140,LIC PEDRO ZORRI,404.0
2,Monterrey,JARDINES DEL PAS MTY,64900,AVE FUNDADORES,5300.0
3,Monterrey,CIUDAD NATURA,64989,AV ABRAHAM LIN,4001.0
4,Monterrey,VILLA LOS PINOS MTY,64770,PINO SILVESTRE,2923.0
...,...,...,...,...,...
351,Escobedo,LA ALIANZA,64103,PRIV PACHUCA,178.0
352,Escobedo,SANTA LUZ,66070,OXFORD,102.0
353,Escobedo,BUENA VISTA,66583,C PELICANO,238.0
354,Escobedo,LOS PARQUES RESIDENCIAL,66023,COBALTO,517.0


In [30]:
productos_porCliente

,Aleatorio,Numero de pedidos,23,29,30,60,66,67,89,91,...,52919,52925,52937,52940,52942,52984,52993,53014,53475,Volumen por pedidos (m^3)
0,0.527082,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.017079
1,0.161062,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
2,0.061546,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
3,0.748310,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.290464
4,0.149171,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,0.502875,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000957
352,0.905077,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.174686
353,0.865957,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.276000
354,0.016917,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [31]:
simulacion_clientes = pd.concat([clientes, productos_porCliente], axis=1)
simulacion_clientes

,ciudad,Colonia,codigo postal,Calle,numero de casa,Aleatorio,Numero de pedidos,23,29,30,...,52919,52925,52937,52940,52942,52984,52993,53014,53475,Volumen por pedidos (m^3)
0,Monterrey,FIERRO MTY,64590,JESUS M GARZA,4026.0,0.527082,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.017079
1,Monterrey,VALLE DE SANTA LUCIA MTY,64140,LIC PEDRO ZORRI,404.0,0.161062,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
2,Monterrey,JARDINES DEL PAS MTY,64900,AVE FUNDADORES,5300.0,0.061546,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
3,Monterrey,CIUDAD NATURA,64989,AV ABRAHAM LIN,4001.0,0.748310,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0.290464
4,Monterrey,VILLA LOS PINOS MTY,64770,PINO SILVESTRE,2923.0,0.149171,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,Escobedo,LA ALIANZA,64103,PRIV PACHUCA,178.0,0.502875,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000957
352,Escobedo,SANTA LUZ,66070,OXFORD,102.0,0.905077,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0.174686
353,Escobedo,BUENA VISTA,66583,C PELICANO,238.0,0.865957,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0.276000
354,Escobedo,LOS PARQUES RESIDENCIAL,66023,COBALTO,517.0,0.016917,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


### Limpieza y filtracion de pedidos

Como existen clientes que no piden ningun pedido podemos reducir la matriz generada en la simulacion

In [32]:
#Eliminamos los clientes que hicieron 0 pedidos
simulacion_clientes =  simulacion_clientes[simulacion_clientes["Numero de pedidos"] != 0]

In [33]:
simulacion_clientes

,ciudad,Colonia,codigo postal,Calle,numero de casa,Aleatorio,Numero de pedidos,23,29,30,...,52919,52925,52937,52940,52942,52984,52993,53014,53475,Volumen por pedidos (m^3)
0,Monterrey,FIERRO MTY,64590,JESUS M GARZA,4026.0,0.527082,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.017079
3,Monterrey,CIUDAD NATURA,64989,AV ABRAHAM LIN,4001.0,0.748310,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0.290464
6,Monterrey,FOME 6 F S T D M MTY,64233,CALLE JOSE ANTO,7319.0,0.869673,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0.001022
7,Monterrey,URBI VILLA DEL REY,64102,LAGO TITICACA,209.0,0.762986,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0.006190
8,Monterrey,PORTAL DEL FRAILE,66064,FRAY JOSE,729.0,0.628113,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.214957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,Escobedo,BUSTAMANTE B,65150,BUGAMBILIAS,435.0,0.889242,2,0,0,0,...,0,0,0,0,0,0,0,0,0,1.591463
351,Escobedo,LA ALIANZA,64103,PRIV PACHUCA,178.0,0.502875,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000957
352,Escobedo,SANTA LUZ,66070,OXFORD,102.0,0.905077,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0.174686
353,Escobedo,BUENA VISTA,66583,C PELICANO,238.0,0.865957,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0.276000


### Coordenadas

In [40]:
from geopy.geocoders import Bing

In [42]:
def quitar_mty(texto):
    if 'MTY' in texto:
        return texto.replace('MTY', '')
    else:
        return texto

# Aplicar la función a la columna 'Ciudad'
simulacion_clientes['Colonia'] = simulacion_clientes['Colonia'].apply(quitar_mty)

# Mostrar el DataFrame resultante
simulacion_clientes

C:\Users\Ruben\AppData\Local\Temp\ipykernel_21080\4024405111.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simulacion_clientes['Colonia'] = simulacion_clientes['Colonia'].apply(quitar_mty)


,ciudad,Colonia,codigo postal,Calle,numero de casa,Aleatorio,Numero de pedidos,23,29,30,...,52919,52925,52937,52940,52942,52984,52993,53014,53475,Volumen por pedidos (m^3)
0,Monterrey,FIERRO,64590,JESUS M GARZA,4026.0,0.527082,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.017079
3,Monterrey,CIUDAD NATURA,64989,AV ABRAHAM LIN,4001.0,0.748310,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0.290464
6,Monterrey,FOME 6 F S T D M,64233,CALLE JOSE ANTO,7319.0,0.869673,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0.001022
7,Monterrey,URBI VILLA DEL REY,64102,LAGO TITICACA,209.0,0.762986,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0.006190
8,Monterrey,PORTAL DEL FRAILE,66064,FRAY JOSE,729.0,0.628113,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.214957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,Escobedo,BUSTAMANTE B,65150,BUGAMBILIAS,435.0,0.889242,2,0,0,0,...,0,0,0,0,0,0,0,0,0,1.591463
351,Escobedo,LA ALIANZA,64103,PRIV PACHUCA,178.0,0.502875,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000957
352,Escobedo,SANTA LUZ,66070,OXFORD,102.0,0.905077,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0.174686
353,Escobedo,BUENA VISTA,66583,C PELICANO,238.0,0.865957,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0.276000


In [43]:
def obtener_coordenadas(tabla):
    coords = []
    geolocator = Bing(api_key='Ago97Rd3jwxWgFxgUWwgUrZ7S21qlE8ok8UvjoEnaXP2nl2Uh8NHqokudoSjLJzI')
    for i in range(tabla.shape[0]):
        city = tabla.iloc[i,0]
        colony = tabla.iloc[i,1]
        postal_code = tabla.iloc[i,2]
        street_name = tabla.iloc[i,3]
        street_number = tabla.iloc[i,4]

        query = f'{street_name}, {street_number}, {colony}, {postal_code}, {city}, Nuevo León, México'
    
        location = geolocator.geocode(query)

        if location:
            latitude = location.latitude
            longitude = location.longitude
            #print(f'Latitude: {latitude}, Longitude: {longitude}')
            coords.append((latitude,longitude))
        else:
            coords.append((0,0))

    return coords

In [44]:
coordenadas = obtener_coordenadas(simulacion_clientes)
coordenadas

[(25.68654935, -100.2772068),
 (25.763761, -100.408254),
 (25.79072633, -100.40731977),
 (36.59832921, -121.90723997),
 (25.8003649, -100.4111165),
 (25.743958, -100.406852),
 (25.66469765, -100.3108902),
 (25.6470789, -100.2856387),
 (25.701319, -100.2986849),
 (25.7241053, -100.35542665),
 (25.7325427, -100.3830339),
 (25.74559021, -100.38032532),
 (25.4300542, -100.1507814),
 (25.68476735, -100.31297545),
 (25.7396928, -100.33933485),
 (25.6919478, -100.30579265),
 (25.66255492, -100.38005504),
 (25.6620275, -100.3009203),
 (25.75633295, -100.36210045),
 (25.66469765, -100.3108902),
 (25.7815921, -100.3799292),
 (25.7350516, -100.3346422),
 (25.683505, -100.325643),
 (25.62057504, -100.26941675),
 (25.7548092, -100.4059584),
 (25.6344434, -100.29773675),
 (25.7396928, -100.33933485),
 (25.77953605, -100.40065487),
 (25.7727002, -100.3915849),
 (25.67532819, -100.37073293),
 (25.68483625, -100.3034782),
 (25.62761028, -100.29564229),
 (25.6487866, -100.1665832),
 (25.7225776, -100.36

In [45]:
def return_latitude(x):
    return x[0]

def return_longitude(x):
    return x[1]

In [46]:
simulacion_clientes["Coordenadas"] = coordenadas
simulacion_clientes["Latitud"] = simulacion_clientes["Coordenadas"].apply(return_latitude)
simulacion_clientes["Longitud"] = simulacion_clientes["Coordenadas"].apply(return_longitude)

C:\Users\Ruben\AppData\Local\Temp\ipykernel_21080\3354209641.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simulacion_clientes["Coordenadas"] = coordenadas
C:\Users\Ruben\AppData\Local\Temp\ipykernel_21080\3354209641.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simulacion_clientes["Latitud"] = simulacion_clientes["Coordenadas"].apply(return_latitude)
C:\Users\Ruben\AppData\Local\Temp\ipykernel_21080\3354209641.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [48]:
clientes_index = simulacion_clientes[(simulacion_clientes['Latitud'] < 25) | (simulacion_clientes['Latitud'] > 26)].index
simulacion_clientes = simulacion_clientes.drop(clientes_index)
simulacion_clientes.shape

(223, 4772)

In [49]:
simulacion_clientes

,ciudad,Colonia,codigo postal,Calle,numero de casa,Aleatorio,Numero de pedidos,23,29,30,...,52940,52942,52984,52993,53014,53475,Volumen por pedidos (m^3),Coordenadas,Latitud,Longitud
0,Monterrey,FIERRO,64590,JESUS M GARZA,4026.0,0.527082,1,0,0,0,...,0,0,0,0,0,0,0.017079,"(25.68654935, -100.2772068)",25.686549,-100.277207
3,Monterrey,CIUDAD NATURA,64989,AV ABRAHAM LIN,4001.0,0.748310,2,0,0,0,...,0,0,0,0,0,0,0.290464,"(25.763761, -100.408254)",25.763761,-100.408254
6,Monterrey,FOME 6 F S T D M,64233,CALLE JOSE ANTO,7319.0,0.869673,2,0,0,0,...,0,0,0,0,0,0,0.001022,"(25.79072633, -100.40731977)",25.790726,-100.407320
8,Monterrey,PORTAL DEL FRAILE,66064,FRAY JOSE,729.0,0.628113,1,0,0,0,...,0,0,0,0,0,0,0.214957,"(25.8003649, -100.4111165)",25.800365,-100.411117
9,Monterrey,CUMBRES 3 SEC,64610,HACIENDA DEL CA,310.0,0.485124,1,0,0,0,...,0,0,0,0,0,0,0.001254,"(25.743958, -100.406852)",25.743958,-100.406852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,Escobedo,BUSTAMANTE B,65150,BUGAMBILIAS,435.0,0.889242,2,0,0,0,...,0,0,0,0,0,0,1.591463,"(25.7864574, -100.2671927)",25.786457,-100.267193
351,Escobedo,LA ALIANZA,64103,PRIV PACHUCA,178.0,0.502875,1,0,0,0,...,0,0,0,0,0,0,0.000957,"(25.78061433, -100.40800749)",25.780614,-100.408007
352,Escobedo,SANTA LUZ,66070,OXFORD,102.0,0.905077,3,0,0,0,...,0,0,0,0,0,0,0.174686,"(25.7924983, -100.3452586)",25.792498,-100.345259
353,Escobedo,BUENA VISTA,66583,C PELICANO,238.0,0.865957,2,0,0,0,...,0,0,0,0,0,0,0.276000,"(25.845431, -100.368877)",25.845431,-100.368877


# Exportación de datos

In [50]:
# Export the DataFrame to a CSV file
simulacion_clientes.to_csv('simulacion_clientes.csv', index=False)  # Set index=False to exclude the DataFrame index